In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

target_user_id = 4

user_ratings = ratings[ratings['userId'] == target_user_id]
data = pd.merge(user_ratings, movies, on='movieId')

user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating', fill_value=0)

user_similarity = cosine_similarity(user_item_matrix)

target_user_watch_history = user_item_matrix.loc[target_user_id].values.reshape(1, -1)
user_similarity_to_target = cosine_similarity(target_user_watch_history, user_item_matrix)

similar_users = list(user_item_matrix.index[np.argsort(user_similarity_to_target[0])[::-1]])[1:]

def get_top_n_recommendations(user_item_matrix, similar_users, n=5):
    aggregated_watch_history = user_item_matrix.loc[similar_users].sum(axis=0)
    unwatched_movies = aggregated_watch_history.index[aggregated_watch_history == 0]
    top_n_recommendations = unwatched_movies[:n]
    return top_n_recommendations

recommendations = get_top_n_recommendations(user_item_matrix, similar_users)

print(f"Top 5 recommendations for User ID {target_user_id}:")
for i, movie_title in enumerate(recommendations, start=1):
    print(f"{i}. {movie_title}")


Top 5 recommendations for User ID 4:
1. 12 Angry Men (1957)
2. Adventures of Priscilla, Queen of the Desert, The (1994)
3. Afterglow (1997)
4. Airplane! (1980)
5. Aladdin (1992)
